In [2]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
import pandas as pd

import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Activation, Dropout, Flatten, Dense, merge, Reshape, Permute, Multiply, Dot,dot, Concatenate, Add
from keras.layers import Input
from keras import backend as K
from keras.engine.topology import Layer
import keras as keras

# packages for learning from crowds
from crowd_layer.crowd_layers import CrowdsClassification, MaskedMultiCrossEntropy, CrowdsClassificationSModel, \
    CrowdsClassificationCModelSingleWeight, CrowdsClassificationCModel, MaskedMultiCrossEntropyCosSim, \
    MaskedMultiCrossEntropyBaseChannel, MaskedMultiCrossEntropyBaseChannelConst, CrowdsClassificationSModelChannelMatrix, \
    MaskedMultiCrossEntropyCurriculumChannelMatrix
from crowd_layer.crowd_aggregators import CrowdsCategoricalAggregator

/home/yajingyang/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# prevent tensorflow from allocating the entire GPU memory at once
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

In [4]:
# Prepare data
def load_data(filename):
    f = open(filename, 'rb')
    data = np.load(f)
    f.close()
    return data

def one_hot(target, n_classes):
    targets = np.array([target]).reshape(-1)
    one_hot_targets = np.eye(n_classes)[targets]
    return one_hot_targets

def get_data(DATA_PATH, N_CLASSES):
    
    print("\nLoading train data...")
    # images processed by VGG16
    data_train_vgg16 = load_data(DATA_PATH+"data_train_vgg16.npy")
    print(data_train_vgg16.shape)

    # ground truth labels
    labels_train = load_data(DATA_PATH+"labels_train.npy")
    print(labels_train.shape)

    # labels obtained from majority voting
    labels_train_mv = load_data(DATA_PATH+"labels_train_mv.npy")
    print(labels_train_mv.shape)

#     # labels obtained by using the approach by Dawid and Skene
#     labels_train_ds = load_data(DATA_PATH+"labels_train_DS.npy")
#     print(labels_train_ds.shape)

    # data from Amazon Mechanical Turk
    print("\nLoading AMT data...")
    answers = load_data(DATA_PATH+"answers.npy")
    print(answers.shape)
    N_ANNOT = answers.shape[1]
    print("N_CLASSES:", N_CLASSES)
    print("N_ANNOT:", N_ANNOT)

    # load test data
    print("\nLoading test data...")

    # images processed by VGG16
    data_test_vgg16 = load_data(DATA_PATH+"data_test_vgg16.npy")
    print(data_test_vgg16.shape)

    # test labels
    labels_test = load_data(DATA_PATH+"labels_test.npy")
    print(labels_test.shape)

    print("\nLoading validation data...")
    # images processed by VGG16
    data_valid_vgg16 = load_data(DATA_PATH+"data_valid_vgg16.npy")
    print(data_valid_vgg16.shape)

    # validation labels
    labels_valid = load_data(DATA_PATH+"labels_valid.npy")
    print(labels_valid.shape)

    labels_train_bin = one_hot(labels_train, N_CLASSES)
    labels_train_mv_bin = one_hot(labels_train_mv, N_CLASSES)
#     labels_train_ds_bin = one_hot(labels_train_ds, N_CLASSES)
#     print(labels_train_ds_bin.shape)
    labels_test_bin = one_hot(labels_test, N_CLASSES)
    labels_valid_bin = one_hot(labels_valid, N_CLASSES)


    answers_bin_missings = []
    for i in range(len(answers)):
        row = []
        for r in range(N_ANNOT):
            if answers[i,r] == -1:
                row.append(-1 * np.ones(N_CLASSES))
            else:
                row.append(one_hot(answers[i,r], N_CLASSES)[0,:])
        answers_bin_missings.append(row)
    answers_bin_missings = np.array(answers_bin_missings).swapaxes(1,2)

    answers_test_bin_missings = np.zeros((len(labels_test), N_CLASSES))
    answers_test_bin_missings[np.arange(len(labels_test)), labels_test] = 1
    answers_test_bin_missings = np.repeat(answers_test_bin_missings.reshape([len(labels_test),N_CLASSES,1]), N_ANNOT, axis=2)

    answers_valid_bin_missings = np.zeros((len(labels_valid), N_CLASSES))
    answers_valid_bin_missings[np.arange(len(labels_valid)), labels_valid] = 1
    answers_valid_bin_missings = np.repeat(answers_valid_bin_missings.reshape([len(labels_valid),N_CLASSES,1]), N_ANNOT, axis=2)
    
    x = {'train': data_train_vgg16, 'test': data_test_vgg16, 'val': data_valid_vgg16}
    y_gt = {'train': labels_train_bin, 'test': labels_test_bin, 'val': labels_valid_bin}
    y_annot = {'train': answers_bin_missings, 'test': answers_test_bin_missings, 'val': answers_valid_bin_missings, 'mv':labels_train_mv_bin}
    return x, y_gt, y_annot

In [5]:
# Build model
def eval(model,x_test, y_test):
    print('Test dataset results: ')
    print(dict(zip(model.metrics_names,model.evaluate(x_test,y_test, verbose=False))))


def get_trace(model):

    channel_matrix = model.get_weights()[-1]
    channel_matrix_trace = tf.trace(K.permute_dimensions(channel_matrix, [2, 0, 1]))
    channel_matrix_trace_arr = K.eval(channel_matrix_trace)
    return channel_matrix_trace_arr


def print_single_loss(model):
    import matplotlib.pyplot as plt

    # list all data in history
    print(model.history.keys())
    # summarize history for accuracy
    plt.plot(model.history['baseline_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.show()
    # summarize history for loss
    plt.plot(model.history['baseline_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.show()


def print_history(df, title):
    import matplotlib.pyplot as plt
    # Make a data frame
    df['x'] = range(df.shape[0])

    # style
    plt.style.use('seaborn-darkgrid')

    # create a color palette
    palette = plt.get_cmap('Set1')

    # multiple line plot
    num = 0
    for column in df.drop('x', axis=1):
        num += 1
        plt.plot(df['x'], df[column], marker='', color=palette(num), linewidth=1, alpha=0.9, label=column)

    # Add legend
    plt.legend(loc=2, ncol=2)

    # Add titles
    plt.title(title, loc='left', fontsize=12, fontweight=0, color='orange')
    plt.xlabel("Time")
    plt.ylabel("Score")
    plt.savefig(title+'.png')

In [6]:
def build_base_model(train_data_shape, N_CLASSES):
    base_model = Sequential()
    base_model.add(Flatten(input_shape=train_data_shape[1:]))
    base_model.add(Dense(128, activation='relu'))
    base_model.add(Dropout(0.4))
    base_model.add(Dense(64, activation='relu'))
    base_model.add(Dropout(0.4))
    base_model.add(Dense(N_CLASSES))
    base_model.add(Activation("softmax"))
    base_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return base_model

In [7]:
def majority_vote(x, y_gt, y_annot, N_CLASSES):
    train_data_shape = x['train'].shape
    N_ANNOT = y_annot['train'].shape[2]
    baseline_model = build_base_model(train_data_shape, N_CLASSES)

#     eval(baseline_model, x['test'], y_test=y_gt['test'])
    history = baseline_model.fit(x['train'], y_annot['mv'], epochs=N_EPOCHS, shuffle=True,
                              batch_size=BATCH_SIZE, verbose=0)
    eval(baseline_model, x['test'], y_test=y_gt['test'])
    return history

In [8]:
def baseline_gt(x, y_gt, N_CLASSES):
    train_data_shape = x['train'].shape
    baseline_model = build_base_model(train_data_shape, N_CLASSES)

#     eval(baseline_model, x['test'], y_test=y_gt['test'])
    history = baseline_model.fit(x['train'], y_gt['train'], epochs=N_EPOCHS, shuffle=True,
                                 batch_size=BATCH_SIZE, verbose=0)
    eval(baseline_model, x['test'], y_test=y_gt['test'])
    return history

In [9]:
def build_base_crowd_model(train_data_shape, N_CLASSES, N_ANNOT, softmax, trace):
    hidden_layers = Sequential()
    hidden_layers.add(Flatten(input_shape=train_data_shape[1:]))
    hidden_layers.add(Dense(128, activation='relu'))
    hidden_layers.add(Dropout(0.4))
    hidden_layers.add(Dense(64, activation='relu'))
    hidden_layers.add(Dropout(0.4))

    train_inputs = Input(shape=(train_data_shape[1:]))
    last_hidden = hidden_layers(train_inputs)
    baseline_output = Dense(N_CLASSES, activation='softmax', name='baseline')(last_hidden)

    if softmax:
        channel_layer = CrowdsClassificationSModelChannelMatrix(N_CLASSES, N_ANNOT)
        channeled_output = channel_layer([last_hidden, baseline_output])
    else:
        channel_layer = CrowdsClassification(N_CLASSES, N_ANNOT)
        channeled_output = channel_layer(baseline_output)



    model = Model(inputs=train_inputs, outputs=[channeled_output, baseline_output])

    if trace:
        loss = MaskedMultiCrossEntropyCurriculumChannelMatrix(model, 1, 1).loss
    else:
        loss = MaskedMultiCrossEntropy().loss

    # compile model with masked loss and train
    model.compile(optimizer='adam',
                         loss=[loss, 'categorical_crossentropy'],
                         loss_weights=[1, 0],
                         metrics=['accuracy']
                        )
    return model

In [10]:
def crowd_model(x, y_gt, y_annot, N_CLASSES, softmax, trace):
    train_data_shape = x['train'].shape
    N_ANNOT = y_annot['train'].shape[2]

    model = build_base_crowd_model(train_data_shape, N_CLASSES, N_ANNOT, softmax, trace)    
    
    history = model.fit(x['train'], [y_annot['train'], y_gt['train']], epochs=N_EPOCHS, shuffle=True,
                              batch_size=BATCH_SIZE, verbose=0)
    trace_arr = get_trace(model)
    eval(model, x['test'], y_test=[y_annot['test'], y_gt['test']])
    return history, trace_arr

In [11]:
def crowd_model_pretrain_with_clean_data(x, y_gt, y_annot, N_CLASSES, softmax, trace):
    train_data_shape = x['train'].shape
    N_ANNOT = y_annot['train'].shape[2]

    hidden_layers = Sequential()
    hidden_layers.add(Flatten(input_shape=train_data_shape[1:]))
    hidden_layers.add(Dense(128, activation='relu'))
    hidden_layers.add(Dropout(0.4))
    hidden_layers.add(Dense(64, activation='relu'))
    hidden_layers.add(Dropout(0.4))

    train_inputs = Input(shape=(train_data_shape[1:]))
    last_hidden = hidden_layers(train_inputs)
    baseline_output = Dense(N_CLASSES, activation='softmax', name='baseline')(last_hidden)

    if softmax:
        channel_layer = CrowdsClassificationSModelChannelMatrix(N_CLASSES, N_ANNOT)
        channeled_output = channel_layer([last_hidden, baseline_output])
    else:
        channel_layer = CrowdsClassification(N_CLASSES, N_ANNOT)
        channeled_output = channel_layer(baseline_output)

    model = Model(inputs=train_inputs, outputs=[channeled_output, baseline_output])

    if trace:
        loss = MaskedMultiCrossEntropyCurriculumChannelMatrix(model, 1, 1).loss
    else:
        loss = MaskedMultiCrossEntropy().loss

    # compile model with masked loss and train
    model.compile(optimizer='adam',
                         loss=[loss, 'categorical_crossentropy'],
                         loss_weights=[1, 0],
                         metrics=['accuracy']
                        )
    
    history = model.fit(x['train'], [y_annot['train'], y_gt['train']], epochs=N_EPOCHS, shuffle=True,
                              batch_size=BATCH_SIZE, verbose=0)
    trace_arr = get_trace(model)
    eval(model, x['test'], y_test=[y_annot['test'], y_gt['test']])
    return history, trace_arr

## Train with LabelMe data

In [26]:
NUM_RUNS = 20
N_CLASSES = 8
BATCH_SIZE = 64
N_EPOCHS = 35
W = 0

DATA_PATH = "/home/yajingyang/Downloads/LabelMe/prepared/"
x, y_gt, y_annot = get_data(DATA_PATH, N_CLASSES)

N_ANNOT = y_annot['train'].shape[2]

for clean_percent in [0.01, 0.02, 0.03, 0.04, 0.05]:
    x_sample = x.copy()
    x_sample['train'] = x_sample['train'][:round(x_sample['train'].shape[0]*clean_percent)]
    y_gt_sample = y_gt.copy()
    y_gt_sample['train'] = y_gt_sample['train'][:round(y_gt_sample['train'].shape[0]*clean_percent)]
    y_annot_mix = y_annot.copy()
    y_annot_mix['train'][:round(y_annot_mix['train'].shape[0]*clean_percent)] \
        = np.reshape(np.repeat(y_gt['train'][:round(y_annot_mix['train'].shape[0]*clean_percent)], N_ANNOT, axis=1)
                     , (-1, N_CLASSES, N_ANNOT))
    print(x_sample['train'].shape, y_gt_sample['train'].shape, y_annot_mix['train'].shape)

    prefix = 'pct_%.2f_'%clean_percent
    loss_csv = prefix + 'loss.csv'
    acc_csv = prefix + 'acc.csv'
    trace_csv = prefix + 'trace.csv'
    acc_df = pd.DataFrame()
    loss_df = pd.DataFrame()
    trace_df = pd.DataFrame()

    print('\nBaseline model with %.2f clean data' % (clean_percent))
    for i in range(NUM_RUNS):
        clean_base_acc_df = pd.DataFrame()
        clean_base_loss_df = pd.DataFrame()
        clean_history = baseline_gt(x_sample, y_gt_sample, N_CLASSES)
        clean_base_acc_df.loc[:, i] = clean_history.history['acc']
        clean_base_loss_df.loc[:, i] = clean_history.history['loss']
            
    print('\nCrowd noise adaptation model with %.2f clean data' % (clean_percent))
    for i in range(NUM_RUNS):
        acc_df = pd.DataFrame()
        loss_df = pd.DataFrame()
        history, trace_arr = crowd_model(x, y_gt, y_annot_mix, N_CLASSES, False, True)
        acc_df.loc[:, i] = history.history['baseline_acc']
        loss_df.loc[:, i] = history.history['baseline_loss']
        
    print('\nCrowd noise adaptation model with %.2f clean data' % (clean_percent))
    for i in range(NUM_RUNS):
        acc_df = pd.DataFrame()
        loss_df = pd.DataFrame()
        history, trace_arr = crowd_model_pretrain_with_clean_data(x, y_gt, y_annot_mix, N_CLASSES, False, True)
        acc_df.loc[:, i] = history.history['baseline_acc']
        loss_df.loc[:, i] = history.history['baseline_loss']    

    #     clean_base_acc_df.to_csv('clean_accuracy.csv')
    #     clean_base_loss_df.to_csv('clean_loss.csv')
    #     print_history(clean_acc_df, 'accuracy')
    #     print_history(clean_loss_df, 'loss')

#         acc_df.to_csv(acc_csv)
#         loss_df.to_csv(loss_csv)
#         trace_df.to_csv(trace_csv)

#     print_history(trace_crowd_acc_df, 'accuracy')
#     print_history(trace_crowd_loss_df, 'loss')
# acc_df.loc[:, str(b)] = history.history['baseline_acc']
# loss_df.loc[:, str(b)] = history.history['baseline_loss']

# acc_df.to_csv('accuracy.csv')
# loss_df.to_csv('loss.csv')
# print_history(acc_df, 'accuracy')
# print_history(loss_df, 'loss')


Loading train data...
(10000, 4, 4, 512)
(10000,)
(10000,)

Loading AMT data...
(10000, 59)
N_CLASSES: 8
N_ANNOT: 59

Loading test data...
(1188, 4, 4, 512)
(1188,)

Loading validation data...
(500, 4, 4, 512)
(500,)
(100, 4, 4, 512) (100, 8) (10000, 8, 59)

Baseline model with 0.01 clean data
Test dataset results: 
{'loss': 0.710868588200322, 'acc': 0.7794612794612794}
Test dataset results: 
{'loss': 0.8067635827594333, 'acc': 0.734006734006734}
Test dataset results: 
{'loss': 0.7348099656478323, 'acc': 0.76010101010101}
Test dataset results: 
{'loss': 0.7108175661226716, 'acc': 0.7634680134680135}
Test dataset results: 
{'loss': 0.6636118900896323, 'acc': 0.7718855218855218}
Test dataset results: 
{'loss': 0.6636052964512347, 'acc': 0.7777777777777778}
Test dataset results: 
{'loss': 0.7271905772212378, 'acc': 0.7643097643097643}
Test dataset results: 
{'loss': 0.7929752127891437, 'acc': 0.7483164983164983}
Test dataset results: 
{'loss': 0.6596275058258262, 'acc': 0.795454545454545

(8,)
Test dataset results: 
{'loss': -7.278589845907809, 'crowds_classification_32_loss': -7.278589845907809, 'baseline_loss': 0.9449352099613509, 'crowds_classification_32_acc': 0.02241161616161616, 'baseline_acc': 0.8409090909090909}
Tensor("baseline_36/Softmax:0", shape=(?, 8), dtype=float32)
<tf.Variable 'crowds_classification_33/CrowdLayer:0' shape=(8, 8, 59) dtype=float32_ref>
Tensor("crowds_classification_33/Reshape_2:0", shape=(?, 8, 59), dtype=float32)
(8,)
Test dataset results: 
{'loss': -7.271381098814685, 'crowds_classification_33_loss': -7.271381098814685, 'baseline_loss': 1.1601700496793999, 'crowds_classification_33_acc': 0.021780303030303032, 'baseline_acc': 0.8409090909090909}
Tensor("baseline_37/Softmax:0", shape=(?, 8), dtype=float32)
<tf.Variable 'crowds_classification_34/CrowdLayer:0' shape=(8, 8, 59) dtype=float32_ref>
Tensor("crowds_classification_34/Reshape_2:0", shape=(?, 8, 59), dtype=float32)
(8,)
Test dataset results: 
{'loss': -7.054001991194908, 'crowds_cl

Test dataset results: 
{'loss': -7.364592542551985, 'crowds_classification_46_loss': -7.364592542551985, 'baseline_loss': 0.8551102845556406, 'crowds_classification_46_acc': 0.02220117845117845, 'baseline_acc': 0.8661616161616161}
Tensor("baseline_50/Softmax:0", shape=(?, 8), dtype=float32)
<tf.Variable 'crowds_classification_47/CrowdLayer:0' shape=(8, 8, 59) dtype=float32_ref>
Tensor("crowds_classification_47/Reshape_2:0", shape=(?, 8, 59), dtype=float32)
(8,)
Test dataset results: 
{'loss': -7.39833929241707, 'crowds_classification_47_loss': -7.39833929241707, 'baseline_loss': 0.7308589501011653, 'crowds_classification_47_acc': 0.020412457912457913, 'baseline_acc': 0.8712121212121212}
Tensor("baseline_51/Softmax:0", shape=(?, 8), dtype=float32)
<tf.Variable 'crowds_classification_48/CrowdLayer:0' shape=(8, 8, 59) dtype=float32_ref>
Tensor("crowds_classification_48/Reshape_2:0", shape=(?, 8, 59), dtype=float32)
(8,)
Test dataset results: 
{'loss': -7.37464823546233, 'crowds_classifica

(8,)
Test dataset results: 
{'loss': -7.196885319269867, 'crowds_classification_60_loss': -7.196885319269867, 'baseline_loss': 3.0505536702345517, 'crowds_classification_60_acc': 0.0, 'baseline_acc': 0.6877104377104377}
Tensor("baseline_64/Softmax:0", shape=(?, 8), dtype=float32)
<tf.Variable 'crowds_classification_61/CrowdLayer:0' shape=(8, 8, 59) dtype=float32_ref>
Tensor("crowds_classification_61/Reshape_2:0", shape=(?, 8, 59), dtype=float32)
(8,)
Test dataset results: 
{'loss': -7.372950984171344, 'crowds_classification_61_loss': -7.372950984171344, 'baseline_loss': 0.8291678223947082, 'crowds_classification_61_acc': 0.0, 'baseline_acc': 0.8358585858585859}
Tensor("baseline_65/Softmax:0", shape=(?, 8), dtype=float32)
<tf.Variable 'crowds_classification_62/CrowdLayer:0' shape=(8, 8, 59) dtype=float32_ref>
Tensor("crowds_classification_62/Reshape_2:0", shape=(?, 8, 59), dtype=float32)
(8,)
Test dataset results: 
{'loss': -7.338504561671504, 'crowds_classification_62_loss': -7.3385045

(8,)
Test dataset results: 
{'loss': -7.414957521740435, 'crowds_classification_78_loss': -7.414957521740435, 'baseline_loss': 0.680633088379396, 'crowds_classification_78_acc': 0.020728114478114477, 'baseline_acc': 0.8636363636363636}
(400, 4, 4, 512) (400, 8) (10000, 8, 59)

Baseline model with 0.04 clean data
Test dataset results: 
{'loss': 0.4253368493771613, 'acc': 0.8712121212121212}
Test dataset results: 
{'loss': 0.41987885130876645, 'acc': 0.8737373737373737}
Test dataset results: 
{'loss': 0.4638470105500735, 'acc': 0.8627946127946128}
Test dataset results: 
{'loss': 0.4449079075219856, 'acc': 0.8627946127946128}
Test dataset results: 
{'loss': 0.481005335320728, 'acc': 0.8627946127946128}
Test dataset results: 
{'loss': 0.43985537843471423, 'acc': 0.8602693602693603}
Test dataset results: 
{'loss': 0.4417877925515978, 'acc': 0.8661616161616161}
Test dataset results: 
{'loss': 0.45826738426831837, 'acc': 0.867003367003367}
Test dataset results: 
{'loss': 0.4438839841772009, '

{'loss': -7.283840226003217, 'crowds_classification_92_loss': -7.283840226003217, 'baseline_loss': 2.3595549964944924, 'crowds_classification_92_acc': 0.00021043771043771043, 'baseline_acc': 0.7676767676767676}
Tensor("baseline_96/Softmax:0", shape=(?, 8), dtype=float32)
<tf.Variable 'crowds_classification_93/CrowdLayer:0' shape=(8, 8, 59) dtype=float32_ref>
Tensor("crowds_classification_93/Reshape_2:0", shape=(?, 8, 59), dtype=float32)
(8,)
Test dataset results: 
{'loss': -7.446992763365158, 'crowds_classification_93_loss': -7.446992763365158, 'baseline_loss': 0.6541185999933555, 'crowds_classification_93_acc': 0.00042087542087542086, 'baseline_acc': 0.8787878787878788}
Tensor("baseline_97/Softmax:0", shape=(?, 8), dtype=float32)
<tf.Variable 'crowds_classification_94/CrowdLayer:0' shape=(8, 8, 59) dtype=float32_ref>
Tensor("crowds_classification_94/Reshape_2:0", shape=(?, 8, 59), dtype=float32)
(8,)
Test dataset results: 
{'loss': -7.350095731240732, 'crowds_classification_94_loss': 

(8,)
Test dataset results: 
{'loss': -7.465105104928065, 'crowds_classification_106_loss': -7.465105104928065, 'baseline_loss': 0.7332952420010862, 'crowds_classification_106_acc': 0.0, 'baseline_acc': 0.8804713804713805}
Tensor("baseline_110/Softmax:0", shape=(?, 8), dtype=float32)
<tf.Variable 'crowds_classification_107/CrowdLayer:0' shape=(8, 8, 59) dtype=float32_ref>
Tensor("crowds_classification_107/Reshape_2:0", shape=(?, 8, 59), dtype=float32)
(8,)
Test dataset results: 
{'loss': -7.313569924646757, 'crowds_classification_107_loss': -7.313569924646757, 'baseline_loss': 2.526913058627334, 'crowds_classification_107_acc': 0.00010521885521885521, 'baseline_acc': 0.7685185185185185}
Tensor("baseline_111/Softmax:0", shape=(?, 8), dtype=float32)
<tf.Variable 'crowds_classification_108/CrowdLayer:0' shape=(8, 8, 59) dtype=float32_ref>
Tensor("crowds_classification_108/Reshape_2:0", shape=(?, 8, 59), dtype=float32)
(8,)
Test dataset results: 
{'loss': -7.405862819466125, 'crowds_classif

## Dogs and cats

In [ ]:
NUM_RUNS = 20
N_CLASSES = 8
BATCH_SIZE = 64
N_EPOCHS = 35
W = 0

DATA_PATH = "/home/yajingyang/Downloads/LabelMe/prepared/"
x, y_gt, y_annot = get_data(DATA_PATH, N_CLASSES)

N_ANNOT = y_annot['train'].shape[2]

for clean_percent in [0.01, 0.02, 0.03, 0.04, 0.05]:
    x_sample = x.copy()
    x_sample['train'] = x_sample['train'][:round(x_sample['train'].shape[0]*clean_percent)]
    y_gt_sample = y_gt.copy()
    y_gt_sample['train'] = y_gt_sample['train'][:round(y_gt_sample['train'].shape[0]*clean_percent)]
    y_annot_mix = y_annot.copy()
    y_annot_mix['train'][:round(y_annot_mix['train'].shape[0]*clean_percent)] \
        = np.reshape(np.repeat(y_gt['train'][:round(y_annot_mix['train'].shape[0]*clean_percent)], N_ANNOT, axis=1)
                     , (-1, N_CLASSES, N_ANNOT))
    print(x_sample['train'].shape, y_gt_sample['train'].shape, y_annot_mix['train'].shape)

    prefix = 'pct_%.2f_'%clean_percent
    loss_csv = prefix + 'loss.csv'
    acc_csv = prefix + 'acc.csv'
    trace_csv = prefix + 'trace.csv'
    acc_df = pd.DataFrame()
    loss_df = pd.DataFrame()
    trace_df = pd.DataFrame()

    print('\nBaseline model with %.2f clean data' % (clean_percent))
    for i in range(NUM_RUNS):
        clean_base_acc_df = pd.DataFrame()
        clean_base_loss_df = pd.DataFrame()
        clean_history = baseline_gt(x_sample, y_gt_sample, N_CLASSES)
        clean_base_acc_df.loc[:, i] = clean_history.history['acc']
        clean_base_loss_df.loc[:, i] = clean_history.history['loss']
            
    print('\nCrowd noise adaptation model with %.2f clean data' % (clean_percent))
    for i in range(NUM_RUNS):
        acc_df = pd.DataFrame()
        loss_df = pd.DataFrame()
        history, trace_arr = crowd_model(x, y_gt, y_annot_mix, N_CLASSES, False, True)
        acc_df.loc[:, i] = history.history['baseline_acc']
        loss_df.loc[:, i] = history.history['baseline_loss']

## Train with Bird Dataset

### Cactus Wren

In [23]:
NUM_RUNS = 2
N_CLASSES = 2
BATCH_SIZE = 64
N_EPOCHS = 35
W = 0

DATA_PATH = '/home/yajingyang/PycharmProjects/online_crowdsourcing/data/classification/cub_40/images/Cactus Wren/'
x, y_gt, y_annot = get_data(DATA_PATH, N_CLASSES)

N_ANNOT = y_annot['train'].shape[2]

print('\nbaseline model with clean labels:')
for i in range(NUM_RUNS):
    baseline_gt(x, y_gt, N_CLASSES)
    
print('\nbaseline model with majority vote labels:')
for i in range(NUM_RUNS):
    majority_vote(x, y_gt, y_annot, N_CLASSES)
    
print('\ncrowd noise adaptation model:')
for i in range(NUM_RUNS):
    crowd_model(x, y_gt, y_annot, N_CLASSES)



Loading train data...
(3069, 7, 7, 512)
(3069,)
(3069,)

Loading AMT data...
(3069, 31)
N_CLASSES: 2
N_ANNOT: 31

Loading test data...
(176, 7, 7, 512)
(176,)

Loading validation data...
(363, 7, 7, 512)
(363,)

baseline model with clean labels:
Test dataset results: 
{'loss': 4.580449364402077, 'acc': 0.6420454545454546}
Test dataset results: 
{'loss': 2.014761946418069, 'acc': 0.875}
Test dataset results: 
{'loss': 1.6465330990878018, 'acc': 0.7386363636363636}
Test dataset results: 
{'loss': 1.007381016557867, 'acc': 0.9375}

baseline model with majority vote labels:
Test dataset results: 
{'loss': 2.49463267107223, 'acc': 0.7840909090909091}
Test dataset results: 
{'loss': 2.747402613813227, 'acc': 0.8295454545454546}
Test dataset results: 
{'loss': 9.89710773121227, 'acc': 0.24431818181818182}
Test dataset results: 
{'loss': 1.8316018256274136, 'acc': 0.8863636363636364}

crowd noise adaptation model:
inputs:  [<tf.Tensor 'sequential_54/dropout_58/cond/Merge:0' shape=(?, 128) dty

In [19]:
NUM_RUNS = 2
N_CLASSES = 2
BATCH_SIZE = 64
N_EPOCHS = 35
W = 0

DATA_PATH = '/home/yajingyang/PycharmProjects/online_crowdsourcing/data/classification/cub_40/images/Cape May Warbler/'
x, y_gt, y_annot = get_data(DATA_PATH, N_CLASSES)

N_ANNOT = y_annot['train'].shape[2]
    
print('\nbaseline model with majority vote labels:')
for i in range(NUM_RUNS):
    majority_vote(x, y_gt, y_annot, N_CLASSES)
    
print('\nbaseline model with clean labels:')
for i in range(NUM_RUNS):
    baseline_gt(x, y_gt, N_CLASSES)

    
print('\ncrowd noise adaptation model:')
for i in range(NUM_RUNS):
    crowd_model(x, y_gt, y_annot, N_CLASSES)


Loading train data...
(3510, 7, 7, 512)
(3510,)
(3510,)

Loading AMT data...
(3510, 40)
N_CLASSES: 2
N_ANNOT: 40

Loading test data...
(210, 7, 7, 512)
(210,)

Loading validation data...
(405, 7, 7, 512)
(405,)

baseline model with majority vote labels:
Test dataset results: 
{'loss': 3.363896878560384, 'acc': 0.6476190487543741}
Test dataset results: 
{'loss': 3.453891637211754, 'acc': 0.7857142857142857}
Test dataset results: 
{'loss': 10.050633058093844, 'acc': 0.2857142858562015}
Test dataset results: 
{'loss': 3.453877686318897, 'acc': 0.7857142857142857}

baseline model with clean labels:
Test dataset results: 
{'loss': 7.196937047867548, 'acc': 0.4190476207506089}
Test dataset results: 
{'loss': 3.453877686318897, 'acc': 0.7857142857142857}
Test dataset results: 
{'loss': 5.360020560310001, 'acc': 0.5095238098076411}
Test dataset results: 
{'loss': 3.453877686318897, 'acc': 0.7857142857142857}

crowd noise adaptation model:
inputs:  [<tf.Tensor 'sequential_36/dropout_36/cond/Me

In [20]:
NUM_RUNS = 2
N_CLASSES = 2
BATCH_SIZE = 64
N_EPOCHS = 35
W = 0

DATA_PATH = '/home/yajingyang/PycharmProjects/online_crowdsourcing/data/classification/cub_40/images/Evening Grosbeak/'
x, y_gt, y_annot = get_data(DATA_PATH, N_CLASSES)

N_ANNOT = y_annot['train'].shape[2]
    
print('\nbaseline model with majority vote labels:')
for i in range(NUM_RUNS):
    majority_vote(x, y_gt, y_annot, N_CLASSES)
    
print('\nbaseline model with clean labels:')
for i in range(NUM_RUNS):
    baseline_gt(x, y_gt, N_CLASSES)

    
print('\ncrowd noise adaptation model:')
for i in range(NUM_RUNS):
    crowd_model(x, y_gt, y_annot, N_CLASSES)


Loading train data...
(4605, 7, 7, 512)
(4605,)
(4605,)

Loading AMT data...
(4605, 26)
N_CLASSES: 2
N_ANNOT: 26

Loading test data...
(270, 7, 7, 512)
(270,)

Loading validation data...
(540, 7, 7, 512)
(540,)

baseline model with majority vote labels:
Test dataset results: 
{'loss': 5.301747682359483, 'acc': 0.5111111114422481}
Test dataset results: 
{'loss': 2.686349210915742, 'acc': 0.8333333333333334}
Test dataset results: 
{'loss': 8.372794212456103, 'acc': 0.3370370357124894}
Test dataset results: 
{'loss': 2.686349210915742, 'acc': 0.8333333333333334}

baseline model with clean labels:
Test dataset results: 
{'loss': 6.542580353772199, 'acc': 0.4629629620799312}
Test dataset results: 
{'loss': 2.686349210915742, 'acc': 0.8333333333333334}
Test dataset results: 
{'loss': 6.45135940975613, 'acc': 0.4555555558866925}
Test dataset results: 
{'loss': 2.746045853473522, 'acc': 0.8296296296296296}

crowd noise adaptation model:
inputs:  [<tf.Tensor 'sequential_42/dropout_42/cond/Merg

In [1]:
NUM_RUNS = 1
N_CLASSES = 2
BATCH_SIZE = 64
N_EPOCHS = 35
W = 0

DATA_PATH = "/home/yajingyang/Downloads/PetImages/"
x, y_gt, y_annot = get_data(DATA_PATH, N_CLASSES)

N_ANNOT = y_annot['train'].shape[2]

for clean_percent in [0.01, 0.02, 0.03, 0.04, 0.05]:
    x_sample = x.copy()
    x_sample['train'] = x_sample['train'][:round(x_sample['train'].shape[0]*clean_percent)]
    y_gt_sample = y_gt.copy()
    y_gt_sample['train'] = y_gt_sample['train'][:round(y_gt_sample['train'].shape[0]*clean_percent)]
    y_annot_mix = y_annot.copy()
    y_annot_mix['train'][:round(y_annot_mix['train'].shape[0]*clean_percent)] \
        = np.reshape(np.repeat(y_gt['train'][:round(y_annot_mix['train'].shape[0]*clean_percent)], N_ANNOT, axis=1)
                     , (-1, N_CLASSES, N_ANNOT))
    print(x_sample['train'].shape, y_gt_sample['train'].shape, y_annot_mix['train'].shape)

    prefix = 'pct_%.2f_'%clean_percent
    loss_csv = prefix + 'loss.csv'
    acc_csv = prefix + 'acc.csv'
    trace_csv = prefix + 'trace.csv'
    acc_df = pd.DataFrame()
    loss_df = pd.DataFrame()
    trace_df = pd.DataFrame()

    print('\nBaseline model with %.2f clean data' % (clean_percent))
    for i in range(NUM_RUNS):
        clean_base_acc_df = pd.DataFrame()
        clean_base_loss_df = pd.DataFrame()
        clean_history = baseline_gt(x_sample, y_gt_sample, N_CLASSES)
        clean_base_acc_df.loc[:, i] = clean_history.history['acc']
        clean_base_loss_df.loc[:, i] = clean_history.history['loss']

NameError: name 'get_data' is not defined